<a href="https://colab.research.google.com/github/aparajithguha/NLP/blob/master/CNN_for_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import re
from google.colab import drive

In [0]:
try:
  %tensorflow_version 2.x
except:
  pass
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [11]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cols = ["sentiment","id","date","query","user","text"]

In [0]:
import pandas as pd

In [0]:
train_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data files/train.csv",header=None,names=cols,engine="python",encoding="latin1")

In [0]:
test_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data files/test.csv",header=None,names=cols,engine="python",encoding="latin1")

In [0]:
train_data.drop(["id","date","query","user"],
          axis =1,
          inplace=True)

In [0]:
from bs4 import BeautifulSoup

In [0]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']",' ', tweet)
  tweet = re.sub(r" +", " ", tweet)
  return tweet

In [0]:
data_clean = [clean_tweet(tweet) for tweet in train_data.text]

In [51]:
data_labels = train_data.sentiment.values
data_labels

array([0, 0, 0, ..., 1, 1, 1])

In [0]:
data_labels[data_labels==4]=1

In [45]:
set(data_labels)

{0, 1}

# Tokenization

In [0]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(data_clean,target_vocab_size=2**16)

In [0]:
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [0]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)